# Project 1: Market Data Collector & EDA Toolkit

## Overview
This project is the first in my **Quantitative Finance Projects** series.  
It focuses on **collecting, cleaning, and exploring market data** as the foundation for trading strategy research.

The goal is to:
- Download historical price data for a set of tickers
- Perform basic exploratory data analysis (EDA)
- Visualise important metrics such as returns, volatility, and drawdowns

All work is documented in **Jupyter Notebooks** to clearly show methodology, results, and annotations.

---
## Features
- Pull OHLCV data from Yahoo Finance (`yfinance`)
- Compute:
  - Daily returns & log returns
  - Rolling volatility
  - Maximum drawdown
- Generate plots:
  - Price history
  - Rolling volatility
  - Drawdown curves
- Modular structure for easy extension in future projects

---

In [ ]:
import os
import pathlib
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

TICKERS = ["AAPL"]
PERIOD = "5y"
INTERVAL = "1d"

frames = []

for t in TICKERS:
    df = yf.download(t, period=PERIOD, interval=INTERVAL, auto_adjust=True, progress=False)

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    
    df = df.rename_axis("date").reset_index()
    df["ticker"] = t
    frames.append(df)

prices = pd.concat(frames, ignore_index=True)

close = (
    prices[["date", "ticker", "Close"]]
    .pivot(index="date", columns="ticker", values="Close")
    .sort_index())     



(1255, 1)